# Clean AWS Bucket

This notebook pulls all .csv files within the index_data folder from the AWS bucket and assigns them to their counterpart entry from the 'calcrai_metrics.csv' file within this repositories utils folder. This is similar to the process used to clean, standardize, and eventually calculate indicator scores for each Cal-CRAI domain.

This notebook creates a new folder for any files not assigned so the user can identify any discrepencies between 'calcrai_metrics.csv' and metric files in AWS.

Lastly, it identifies any rows within the 'calcrai_metrics.csv' that did not recieve a metric file pulled from AWS.

In [1]:
import pandas as pd
import os
import sys
import warnings
import shutil

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory

In [ ]:
# set-up
bucket_name = 'ca-climate-index'
aws_dir = '3_fair_data/index_data/'

pull_csv_from_directory(bucket_name, aws_dir, output_folder='aws_csvs', search_zipped=False, print_name=False)

In [ ]:
# Define the output folder path
output_folder = 'aws_csvs'

# Count the number of files in the folder
file_count = len([file for file in os.listdir(output_folder) if os.path.isfile(os.path.join(output_folder, file))])

print(f'There are {file_count} files in the {output_folder} folder.')

In [ ]:
meta_csv = r'../utils/calcrai_metrics.csv'
metadata_calculate = pd.read_csv(meta_csv)

In [ ]:
non_metric_folder = 'non_metric_folder'  # Folder to store unmatched files

# Read the CSV and extract the 'File Name' column
metadata_filenames = metadata_calculate['Metric file name'].tolist()  # List of valid file names from the CSV

# Create the non_metric_folder folder if it doesn't exist
if not os.path.exists(non_metric_folder):
    os.makedirs(non_metric_folder)

# Iterate through the files in the output folder
for file_name in os.listdir(output_folder):
    # Full path to the file in the output folder
    file_path = os.path.join(output_folder, file_name)
    
    # If it's a file and not in the metadata list, move it to the non_metric_folder folder
    if os.path.isfile(file_path) and file_name not in metadata_filenames:
        # Move the file to the non_metric_folder folder
        shutil.move(file_path, os.path.join(non_metric_folder, file_name))

print(f"Files that don't match the metadata have been moved to {non_metric_folder}.")

In [ ]:
# Count the number of files in the folder
file_count = len([file for file in os.listdir(non_metric_folder) if os.path.isfile(os.path.join(non_metric_folder, file))])

print(f'There are {file_count} files in the {non_metric_folder} folder.')

In [ ]:
# Define the output folder path
output_folder = 'aws_csvs'

# Count the number of files in the folder
file_count = len([file for file in os.listdir(output_folder) if os.path.isfile(os.path.join(output_folder, file))])

print(f'There are {file_count} files in the {output_folder} folder.')

In [ ]:
# List all files remaining in the aws_csvs folder
remaining_files = os.listdir(output_folder)

# Find the rows in metadata that don't have a corresponding file in aws_csvs
missing_files = [file_name for file_name in metadata_filenames if file_name not in remaining_files]

# Filter the metadata DataFrame to show the rows corresponding to missing files
missing_rows = metadata_calculate[metadata_calculate['Metric file name'].isin(missing_files)]

# Display the missing rows
print(f"These {len(missing_rows)} rows in 'meta_calculate_csv' that don't have a corresponding file in 'aws_csvs':")
display(missing_rows)

# Optionally, save the missing rows to a new CSV
# missing_rows.to_csv('missing_files_in_metadata.csv', index=False)